# Import libraries

In [6]:
import Bio.SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import numpy as np
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow import keras as k
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define functions

In [3]:
def mcc(label_real, label_predicted):
    """Custom Metric for Accuracy and Precision

    Parameters:
    ----------
    label_real: tensorflow.Tensor
        Correct labels
    label_predicted: tensorflow.Tensor
        Predicted labels

    Returns
    -------
    float
        Custom metric
    """
    true_pos = K.sum(K.round(K.clip(label_real * label_predicted, 0, 1)))
    true_neg = K.sum(K.round(K.clip((1 - label_real) * (1 - label_predicted), 0, 1)))
    false_pos = K.sum(K.round(K.clip((1 - label_real) * label_predicted, 0, 1)))
    false_neg = K.sum(K.round(K.clip(label_real * (1 - label_predicted), 0, 1)))
    number = true_pos * true_neg - false_pos * false_neg
    denominator = K.sqrt((true_pos + false_pos) * 
                        (true_pos + false_neg) * 
                        (true_neg + false_pos) * 
                        (true_neg + false_neg))
    return number / (denominator + K.epsilon())

In [4]:
def one_hot_encoding_aa(dataset):
    """One hot encoding of Amino Acid sequences

    Parameters:
    ----------
    dataset: List[str]
        List of Amino Acid sequences

    Returns
    -------
    numpy.ndarray
        One Hot Encoding of the amino acids (has dimensions dataset_size x seq_len x 21)
    """
    integer_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder(categories='auto')
    amino_acids = "ARNDCQEGHILKMFPSTWYV*"
    input_features = []

    # fix the encoded categories
    ie = integer_encoder.fit_transform(list(amino_acids)) #.toarray().reshape(-1, 1)
    ie = np.array(ie).reshape(-1, 1)
    oe = one_hot_encoder.fit_transform(ie)

    for sequence in dataset:
        if type(sequence) == str:
            integer_encoded = integer_encoder.transform(list(sequence)) #.toarray().reshape(-1, 1)
            integer_encoded = np.array(integer_encoded).reshape(-1,1)
            one_hot_encoded = one_hot_encoder.transform(integer_encoded)
            input_features.append(one_hot_encoded.toarray())
            
    input_features = pad_sequences(input_features, padding="post")
    input_features = np.stack(input_features)
    
    return input_features

In [5]:
def one_hot_encoding(dataset, maxlen):
    """One hot encoding of DNA sequences

    Parameters:
    ----------
    dataset: List[str]
        List of DNA sequences
    maxlen: int
        Max length of padded sequences

    Returns
    -------
    numpy.ndarray
        One Hot Encoding of the DNA sequences (has dimensions dataset_size x maxlen x 4)
    """
    integer_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder(categories='auto')
    input_features = []
    for sequence in dataset:
        if type(sequence) == str:
            integer_encoded = integer_encoder.fit_transform(list(sequence)) #.toarray().reshape(-1, 1)
            integer_encoded = np.array(integer_encoded).reshape(-1,1)
            one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
            input_features.append(one_hot_encoded.toarray())

    input_features = pad_sequences(input_features, padding="post", maxlen=maxlen)
    input_features = np.stack(input_features)
    
    return input_features

# Load models and mutant library

In [ ]:
candidates = []
description = []
for record in Bio.SeqIO.parse("Insilico/function_test_4-5_aa.fasta", "fasta"):
    candidat = str(record.seq)
    candidates.append(candidat)
    description.append(record.description)
candidate = one_hot_encoding_aa(candidates)

In [ ]:
candidates = []
description = []
for record in Bio.SeqIO.parse("Insilico/PR_combination_aa.fasta", "fasta"):
    candidat = str(record.seq)
    candidates.append(candidat)
    description.append(record.description)
candidate = [one_hot_encoding_aa(candidates)]

In [ ]:
candidates = []
description = []
for record in Bio.SeqIO.parse("Insilico/function_test_3-4_dna.fasta", "fasta"):
    #candidat = str(record.seq.translate())
    candidates.append(str(record.seq))
    description.append(record.description)
candidate = [one_hot_encoding(candidates)]

In [6]:
candidates = []
description = []
for record in Bio.SeqIO.parse("Insilico/single_mutant_candidates.fasta", "fasta"):
    #candidat = str(record.seq.translate())
    candidates.append(str(record.seq))
    description.append(record.description)
candidate = [one_hot_encoding(candidates)]

In [9]:
# TODO: Optimize the hyperparameters
# https://colab.research.google.com/drive/17E4h5aAOioh5DiTo7MZg4hpL6Z_0FyWr#scrollTo=eiiwjw4yhX0P
# Function to build the models: simple 1D CNN model for binary classification
# Input: one hot encoded sequence list

#[CV 2/3; 1/1] START activation=elu, epochs=23, filters_size=373, num_of_layers=2, optimize=nadam

from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow import keras as k

def cnn_1d(train_features):
    model = Sequential()
    model.add(Conv1D(filters=49, kernel_size=3, input_shape=(1784, 4)))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=49, kernel_size=3, input_shape=(1784, 4)))
    model.add(MaxPooling1D(pool_size=4))
    # model.add(Flatten())
    model.add(Conv1D(filters=49, kernel_size=3, input_shape=(1784, 4)))
    model.add(MaxPooling1D(pool_size=4))
    #model.add(Conv1D(filters=400, kernel_size=3, input_shape=(1784, 4)))
    #model.add(MaxPooling1D(pool_size=4))
    model.add(Flatten())
    model.add(Dense(16, activation='elu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['binary_accuracy', mcc])

    # if you need to convert the model to a multi-class classification model
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

    model.summary()
    return model

In [10]:
dependencies = {
    'mcc': mcc
}

#signal_model = keras.models.load_model('CNN_aa/trained_models/mcc_BC_opt_aaprop_signal_v0.1.h5', compile=False)
#bkg_model = keras.models.load_model('CNN_aa/trained_models/mcc_BC_opt_aaprop_bkg_v0.2.h5', compile=False)

#signal_model = keras.models.load_model('CNN_aa/trained_models/mcc_BC_opt_aaprop_signal_v0.1.h5', compile=False)
background_model = keras.models.load_model('CNN_multilabel/trained_models/binary_classification_CNN1D_bkg_v2.3.h5', compile=False)
#signal_model.load_weights("CNN_multilabel/val_signal_best_weights_4.h5")
signal_model = cnn_1d(1784)
signal_model.load_weights("CNN_multilabel/opt_sig_best.h5")

# with h5py.File('CNN_multilabel/opt_sig_best.h5', 'r') as f:
#     # Iterate through the layers in the model
#     for layer_name in f.keys():
#         layer = f[layer_name]
#         print(layer.values)
#         for i in layer:
#             print(i)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1782, 49)          637       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 445, 49)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 443, 49)           7252      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 110, 49)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 108, 49)           7252      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 27, 49)           0

# Predict samples

In [11]:
signal_prediction = signal_model.predict(candidate)#.flatten().tolist()

bkg_prediction = background_model.predict(candidate)#.flatten().tolist()seq

44566/44566 [==============================] - 213s 5ms/step


In [ ]:

    for idx, row in prediction.iterrows():
        sig_val, bkg_val = row['Signal'], row['Background']
        if round(sig_val) == 0 and round(bkg_val) == 0:
            lslb.append(idx)
        elif round(sig_val) == 0 and round(bkg_val) == 1:
            lshb.append(idx)
        elif round(sig_val) == 1 and round(bkg_val) == 0:
            if not "*" in description[index]:
                hslb.append(idx)
        elif round(sig_val) == 1 and round(bkg_val) == 1:
            hshb.append(idx)

    lslb.sort(key=lambda idx: (prediction.iloc[idx]['Background'], prediction.iloc[idx]['Signal']))
    lshb.sort(key=lambda idx: (prediction.iloc[idx]['Background'], -prediction.iloc[idx]['Signal']))
    hslb.sort(key=lambda idx: (-prediction.iloc[idx]['Background'], prediction.iloc[idx]['Signal']))
    hshb.sort(key=lambda idx: (-prediction.iloc[idx]['Background'], -prediction.iloc[idx]['Signal']))


In [15]:
actual_values=["low", "low", "high", "high", "low",
              "high", "low", "high", "high", "low", "low",
              "low", "low", "low", "high", "high", "low",
              "low", "high", "high", "high"]

In [16]:
signal_prediction

array([[0.36970812],
       [0.5214745 ],
       [0.43355188],
       [0.4899329 ],
       [0.40918002],
       [0.3746195 ],
       [0.42283514],
       [0.5068196 ],
       [0.53170043],
       [0.48283526],
       [0.28146476],
       [0.3872459 ],
       [0.49370536],
       [0.49925408],
       [0.5197781 ],
       [0.6088199 ],
       [0.48644224],
       [0.39418834],
       [0.5129883 ],
       [0.501869  ],
       [0.72036624]], dtype=float32)

In [20]:
# pred_labels = []
# for i in signal_prediction:
#     if i[0] > i[1]:
#         pred = "low"
#     else:
#         pred = "high"
#     pred_labels.append(pred)
    
for i in np.round(signal_prediction):
    if i > 0.52:
        pred = "high"
    else: 
        pred = "low"
    pred_labels.append(pred)
d = pd.DataFrame(list(zip(pred_labels, actual_values)), columns=["predicted", "actual"], index=description)
count = 0
for i in range(21):
    if actual_values[i] != "-":
        if pred_labels[i] == actual_values[i]:
            count += 1
print(count/21)

0.8095238095238095


In [56]:
bkg_predi = []
for i in bkg_prediction:
    bkg_predi.append(i[1])
bkg_pred = pd.DataFrame(bkg_predi, columns=["Background"])#, index=description)
sig_pred = pd.DataFrame(signal_prediction, columns=["Signal"])#, index=description)
#bkg_pred

In [57]:
#prediction = pd.DataFrame(list(zip(pred_labels, actual_values)), columns=["Signal", "actual"], index=description)
#prediction = pd.DataFrame(list(zip(signal_prediction)), columns=["Signal"], index=description)
prediction = pd.concat([sig_pred, bkg_pred], axis=1)

In [58]:
prediction.sort_values('Signal', ascending=False)

,Signal,Background
294554,1.000000,0.997767
295586,1.000000,0.999999
335894,1.000000,1.000000
780206,1.000000,1.000000
295819,1.000000,1.000000
...,...,...
601001,0.011561,0.018716
59279,0.009416,0.955611
183239,0.009305,0.999547
79504,0.008028,0.055246


In [59]:
prediction

,Signal,Background
0,0.294429,0.645030
1,0.145416,0.991693
2,0.433325,1.000000
3,0.550178,0.062479
4,0.474823,0.998845
...,...,...
1426106,0.209240,0.679889
1426107,0.265832,0.608610
1426108,0.372132,0.242968
1426109,0.528983,0.052316


In [60]:
predi = []
for i in prediction["Signal"]:
    if i < 0.52:
        predi.append(i)

In [61]:
signal_sort = prediction.sort_values(['Signal', 'Background'], ascending=[False, False])

In [62]:
hslb = []
hshb = []
lshb = []
lslb = []

for index, row in signal_sort.iterrows():
    if round(row['Background']) == 0:
        if not "*" in description[index]:
            hslb.append(index)
    elif round(row['Background']) == 1:
        hshb.append(index)

for index, row in signal_sort[::-1].iterrows():
    if round(row['Background']) == 0:
        lslb.append(index)
    elif round(row['Background']) == 1:
        lshb.append(index)

In [63]:
print(len(hslb))
print(len(hshb))
print(len(lshb))
print(len(lslb))

603505
677464
677464
748647


In [65]:
signal_sort

,Signal,Background
94052,1.000000,1.000000
142630,1.000000,1.000000
221941,1.000000,1.000000
295819,1.000000,1.000000
335894,1.000000,1.000000
...,...,...
601001,0.011561,0.018716
59279,0.009416,0.955611
183239,0.009305,0.999547
79504,0.008028,0.055246


In [70]:
hslb_f = hslb[:101]
hshb_f = hshb[:100]
lshb_f = lshb[:100]
lslb_f = lslb[:100]

In [420]:
hshb

[11583, 16321, 32547, 131462, 197042]

In [421]:
lshb

[798857, 673755, 561222, 3407, 406260]

In [422]:
lslb

[796378, 886001, 474654, 17161, 453000]

In [72]:
prediction.iloc[hslb_f]

,Signal,Background
1340442,0.999999,0.168831
128262,0.999998,0.309711
632052,0.999997,0.055607
29588,0.999997,0.227455
1224988,0.999985,0.333686
...,...,...
971532,0.997839,0.056769
1322205,0.997815,0.055699
827648,0.997797,0.060096
1314594,0.997796,0.043655


In [424]:
prediction.iloc[hshb]

,Signal,Background
11583,1.0,1.0
16321,1.0,1.0
32547,1.0,1.0
131462,1.0,1.0
197042,1.0,1.0


In [425]:
prediction.iloc[lshb]

,Signal,Background
798857,0.001696,0.847137
673755,0.002086,0.971720
561222,0.002734,0.984453
3407,0.002936,0.999999
406260,0.004137,0.999439


In [426]:
prediction.iloc[lslb]

,Signal,Background
796378,0.000634,0.333993
886001,0.001455,0.000823
474654,0.003202,0.000313
17161,0.003309,0.001406
453000,0.003374,0.000002


In [73]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

fasta_dna = []
fasta_aa = []
for i in hslb_f:
    signal = signal_prediction[i][0]
    bkg = bkg_predi[i]
    dna_read = SeqRecord(Seq(candidates[i]), id=description[i], description=f"{signal}  {bkg}")
    fasta_dna.append(dna_read)
Bio.SeqIO.write(fasta_dna, "high_candidates_dna.fasta", "fasta")


101

In [28]:
for i in range(len(hslb_f)):
    print("hslb", description[hslb_f[i]])

TypeError: list indices must be integers or slices, not str

In [ ]:
fasta_dna = []
fasta_aa = []
for i in range(len(hslb_f)):
    diff = dif(ref_aa, candidates_aa[i])
    description=[]
    for x in diff:
        des = f"{ref_aa[x]}{x + 1}{candidates_aa[i][x]}"
        description.append(des)
    description = "|".join(description)
    dna_read = SeqRecord(Seq(candidates_dna[i]), id = "DmpR_insilico_mutant", description=description)
    fasta_dna.append(dna_read)
    aa_read = SeqRecord(Seq(candidates_aa[i]), id = "DmpR_insilico_mutant", description=description)
    fasta_aa.append(aa_read)
Bio.SeqIO.write(fasta_dna, f"{filename}_dna.fasta", "fasta")
Bio.SeqIO.write(fasta_aa, f"{filename}_aa.fasta", "fasta")
    

In [24]:
for i in range(5):
    #print("hslb", description[hslb[i]])
    print("hshb", description[hshb[i]])
    #print("lshb", description[lshb[i]])
    #print("lslb", description[lslb[i]])

hshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('R246S', 'E8G', 'V134E', 'E135K')
hshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('R246S', 'E8A', 'V134E', 'E135K')
hshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('R246S', 'E8K', 'V134E', 'E135K')
hshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('R246S', 'E133V', 'E8K', 'E135K')
hshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('R246S', 'E133G', 'E8K', 'E135K')


In [27]:
for i in range(5):
    print("lshb", description[lshb[i]])

lshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('T224K', 'Q238H', 'E8G', 'E133V')
lshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('T224K', 'Q238H', 'E8G', 'E133G')
lshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('T224K', 'Q238H', 'E8G', 'V134E')
lshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('T224K', 'Q238H', 'E8A', 'E133V')
lshb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('T224K', 'Q238H', 'E8A', 'E133G')


In [28]:
for i in range(5):
    print("lslb", description[lslb[i]])

lslb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('F14S', 'T224K', 'Q238H', 'E8G')
lslb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('F14S', 'T224K', 'Q238H', 'E8A')
lslb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('F14S', 'Q238H', 'E8G', 'E133V')
lslb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('F14S', 'Q238H', 'E8G', 'E133G')
lslb DmpR_insilico_mutant DmpR_Predicted_residue_combination ('F14S', 'E89V', 'Q238H', 'E8G')


In [477]:
import itertools

letters = ['A', 'G', 'T', 'C']

# Use itertools.product to generate combinations
combinations = [''.join(combination) for combination in itertools.product(letters, repeat=8)]

# Print the combinations
#for combination in combinations:
print(len(combinations))

65536
